In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import math
from datetime import date, timedelta, datetime
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.dates as mdates
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score # Packages for measuring model performance / errors
from tensorflow.keras import Sequential # Deep learning library, used for neural networks
from tensorflow.keras.layers import LSTM, Dense, Dropout # Deep learning classes for recurrent and regular densely-connected layers
from tensorflow.keras.callbacks import EarlyStopping # EarlyStopping during model training
from sklearn.preprocessing import RobustScaler, MinMaxScaler # This Scaler removes the median and scales the data according to the quantile range to normalize the price data 
import seaborn as sns # Visualization

c:\Users\iqmalhafiyy\Anaconda3\envs\forex\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\Users\iqmalhafiyy\Anaconda3\envs\forex\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\Users\iqmalhafiyy\Anaconda3\envs\forex\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\Users\iqmalhafiyy\Anaconda3\envs\forex\lib\site-pack

In [2]:
#USDMYR
main_df = pd.read_csv('USDMYR.csv', parse_dates=True, dayfirst=True,  index_col=0,usecols=['Date','Open','High','Low','Close'])
main_df

,Open,High,Low,Close
Date,,,,
2003-12-01,3.7870,3.8031,3.7857,3.7968
2003-12-02,3.7964,3.8021,3.7890,3.7971
2003-12-03,3.7974,3.8037,3.7892,3.7902
2003-12-04,3.7908,3.8075,3.7830,3.7987
2003-12-05,3.7959,3.8029,3.7884,3.7896
...,...,...,...,...
2022-04-20,4.2530,4.2900,4.2520,4.2530
2022-04-21,4.2790,4.2930,4.2790,4.2790
2022-04-22,4.2885,4.3235,4.2885,4.2885


In [3]:
#check missing values
print(main_df.isnull().sum())

Open     9
High     9
Low      9
Close    9
dtype: int64


In [4]:
main_df_cleaned = main_df.dropna()

In [6]:
# Crude Oil Data
oil_data = pd.read_csv('Crude Oil.csv', parse_dates=True, dayfirst=True,  index_col=0,usecols=['Date','Open_Oil','High_Oil','Low_Oil','Close_Oil'])
print(oil_data)
print(oil_data.isnull().sum())

           Open_Oil High_Oil Low_Oil Close_Oil
Date                                          
2022-04-27   102.11   102.13   100.8    101.03
2022-04-27   101.76   102.99    99.8    102.02
2022-04-26    98.64   102.78   97.06     101.7
2022-04-25   101.38   101.55   95.28     98.54
2022-04-22   104.07   104.22  101.06    102.07
...             ...      ...     ...       ...
2013-08-29   109.53   110.07  107.72     108.8
2013-08-28   109.22   112.24  109.11     110.1
2013-08-27   106.14   109.32  105.88    109.01
2013-08-26   106.91   107.37  105.56    105.92
2013-08-23   105.22   106.94   104.3    106.42

[2187 rows x 4 columns]
Open_Oil     0
High_Oil     0
Low_Oil      0
Close_Oil    0
dtype: int64


In [10]:
#Gold Data
gold_data = pd.read_csv('Gold_Price.csv', parse_dates=True, dayfirst=True, index_col=0, usecols=['Date','Open_Gold','High_Gold','Low_Gold','Close_Gold'])
print(gold_data)
print(gold_data['Open_Gold'].str.contains('-').value_counts()[True])

           Open_Gold High_Gold Low_Gold Close_Gold
Date                                              
2022-04-27    1886.8    1887.3   1880.4     1883.4
2022-04-27    1897.8    1900.6   1882.2     1885.9
2022-04-26    1900.5    1909.1     1894     1901.4
2022-04-25      1925      1925   1889.3     1893.2
2022-04-22    1949.6    1950.4   1930.2       1931
...              ...       ...      ...        ...
2000-09-05     275.8     275.8    275.8      275.8
2000-09-04         -         -        -          -
2000-09-01       277       277      277        277
2000-08-31     274.8     278.3    274.8      278.3
2000-08-30     273.9     273.9    273.9      273.9

[5516 rows x 4 columns]
81


In [12]:
gold_data.drop(gold_data[gold_data['Open_Gold'] == '-'].index, inplace=True)
gold_data

,Open_Gold,High_Gold,Low_Gold,Close_Gold
Date,,,,
2022-04-27,1886.8,1887.3,1880.4,1883.4
2022-04-27,1897.8,1900.6,1882.2,1885.9
2022-04-26,1900.5,1909.1,1894,1901.4
2022-04-25,1925,1925,1889.3,1893.2
2022-04-22,1949.6,1950.4,1930.2,1931
...,...,...,...,...
2000-09-06,274.2,274.2,274.2,274.2
2000-09-05,275.8,275.8,275.8,275.8
2000-09-01,277,277,277,277


In [13]:
USDMYR_Oil = pd.merge(main_df_cleaned, oil_data, how='inner', on='Date')
USDMYR_Oil 

,Open,High,Low,Close,Open_Oil,High_Oil,Low_Oil,Close_Oil
Date,,,,,,,,
2013-08-23,3.3079,3.3116,3.1248,3.2984,105.22,106.94,104.3,106.42
2013-08-26,3.2966,3.3081,3.2859,3.3015,106.91,107.37,105.56,105.92
2013-08-27,3.3092,3.3362,3.2911,3.3277,106.14,109.32,105.88,109.01
2013-08-28,3.3278,3.3385,3.3150,3.3315,109.22,112.24,109.11,110.1
2013-08-29,3.3175,3.3249,3.2916,3.3078,109.53,110.07,107.72,108.8
...,...,...,...,...,...,...,...,...
2022-04-20,4.2530,4.2900,4.2520,4.2530,103.05,104.16,100.7,102.75
2022-04-21,4.2790,4.2930,4.2790,4.2790,102.5,105.42,102.01,103.79
2022-04-22,4.2885,4.3235,4.2885,4.2885,104.07,104.22,101.06,102.07


In [14]:
data_join = pd.merge(USDMYR_Oil, gold_data, how='inner', on='Date')
data_join

,Open,High,Low,Close,Open_Oil,High_Oil,Low_Oil,Close_Oil,Open_Gold,High_Gold,Low_Gold,Close_Gold
Date,,,,,,,,,,,,
2013-08-23,3.3079,3.3116,3.1248,3.2984,105.22,106.94,104.3,106.42,1376.8,1398.2,1373.9,1395.7
2013-08-26,3.2966,3.3081,3.2859,3.3015,106.91,107.37,105.56,105.92,1398.6,1402.8,1391.3,1393
2013-08-27,3.3092,3.3362,3.2911,3.3277,106.14,109.32,105.88,109.01,1403.8,1421.1,1398.7,1420.6
2013-08-28,3.3278,3.3385,3.3150,3.3315,109.22,112.24,109.11,110.1,1417.3,1428,1417.3,1419
2013-08-29,3.3175,3.3249,3.2916,3.3078,109.53,110.07,107.72,108.8,1417.4,1417.4,1404.5,1412.9
...,...,...,...,...,...,...,...,...,...,...,...,...
2022-04-20,4.2530,4.2900,4.2520,4.2530,103.05,104.16,100.7,102.75,1945.5,1953.6,1945.5,1952.3
2022-04-21,4.2790,4.2930,4.2790,4.2790,102.5,105.42,102.01,103.79,1942.5,1944.9,1942.5,1944.9
2022-04-22,4.2885,4.3235,4.2885,4.2885,104.07,104.22,101.06,102.07,1949.6,1950.4,1930.2,1931


In [ ]:
# # We add a prediction column and set dummy values to prepare the data for scaling
# main_df_ext = data_join.copy()
# main_df_ext['Prediction'] = main_df_ext['Close_x']
# main_df_ext

In [15]:
#split data

n = len(data_join)
train_df = data_join[0:int(n*0.7)]
val_df = data_join[int(n*0.7):int(n*0.9)]
test_df = data_join[int(n*0.9):]

In [ ]:
# train_mean = train_df.mean()
# train_std = train_df.std()

# train_df = (train_df - train_mean) / train_std
# val_df = (val_df -train_mean) / train_std
# test_df = (test_df - train_mean) / train_std

In [16]:
class WindowGenerator():
    def __init__(self, input_width, label_width, shift, 
                 train_df=train_df, val_df=val_df, test_df=test_df, 
                 label_columns=None):
        
        # Store the raw data
        self.train_df = train_df
        self.val_df = val_df
        self.test_df = test_df
        
        # Work out the label column indices.
        self.label_columns = label_columns
        if label_columns is not None:
            self.label_columns_indices = {name: i for i, name in 
                                          enumerate(label_columns)}
        self.column_indices = {name: i for i, name in enumerate(train_df.columns)}
        
        # Work out the window parameters.
        self.input_width = input_width
        self.label_width = label_width
        self.shift = shift

        self.total_window_size = input_width + shift

        self.input_slice = slice(0, input_width)
        self.input_indices = np.arange(self.total_window_size)[self.input_slice]

        self.label_start = self.total_window_size - self.label_width
        self.labels_slice = slice(self.label_start, None)
        self.label_indices = np.arange(self.total_window_size)[self.labels_slice]

    def __repr__(self):
        return '\n'.join([
            f'Total window size: {self.total_window_size}',
            f'Input indices: {self.input_indices}',
            f'Label indices: {self.label_indices}',
            f'Label column name(s): {self.label_columns}'])


In [17]:

w1 = WindowGenerator(input_width=30, label_width=1, shift=1, 
                    label_columns=['Close'])
w1

Total window size: 31
Input indices: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29]
Label indices: [30]
Label column name(s): ['Close']

In [18]:
def split_window(self, features):
  inputs = features[:, self.input_slice, :]
  labels = features[:, self.labels_slice, :]
  if self.label_columns is not None:
    labels = tf.stack(
        [labels[:, :, self.column_indices[name]] for name in self.label_columns],
        axis=-1)

  # Slicing doesn't preserve static shape information, so set the shapes
  # manually. This way the `tf.data.Datasets` are easier to inspect.
  inputs.set_shape([None, self.input_width, None])
  labels.set_shape([None, self.label_width, None])

  return inputs, labels

WindowGenerator.split_window = split_window

In [22]:
# Stack three slices, the length of the total window.
example_window = tf.stack([np.array(train_df[:w1.total_window_size]),
                           np.array(train_df[100:100+w1.total_window_size]),
                           np.array(train_df[200:200+w1.total_window_size])])

example_inputs, example_labels = w1.split_window(example_window)

print('All shapes are: (batch, time, features)')
print(f'Window shape: {example_window.shape}')
print(f'Inputs shape: {example_inputs.shape}')
print(f'Labels shape: {example_labels.shape}')

TypeError: Expected binary or unicode string, got 3.3079